# Eerste Example Functions Flows

This notebook is used to parameterize the .toml configuration file used to run the eerste example as a console application. 

In [1]:
import sys

import numpy as np
import pandas as pd
import tomli
import tomli_w
  
DIRECTORY = '/Users/johnkucharski/Documents/source/functionalflows/functionalflows/functionalflows/'
sys.path.append(DIRECTORY)
from utilities import day_of_water_year, liters_per_day_to_m3_per_second

The functional flows program analyzes water rather than calendar years. The days in the calendar year that correspond with the beginning of the Eerste watershed water year is printed below.

In [2]:
first_day_of_wy = pd.to_datetime("5-1-2023").day_of_year
print(f'start of water year is the {first_day_of_wy}st day of the year.')

start of water year is the 121st day of the year.


### Input data

The Eerste example will analyze the input data (df_input) printed to a input.csv file below.

In [3]:
df_simulated = pd.read_csv(DIRECTORY + '/examples/eerste/simulated_inflow.csv', parse_dates=['dates'], index_col='dates')
df_simulated = df_simulated[['flows']]
df_simulated['flows'] = df_simulated['flows'].replace(-9999.0, np.nan)
df_simulated['flows'] = df_simulated['flows'].apply(liters_per_day_to_m3_per_second)
df_input = df_simulated[['flows']]
df_input.head()


,flows
dates,
1997-01-01 07:30:00,5.949074
1997-01-02 07:30:00,1.655093
1997-01-03 07:30:00,0.628472
1997-01-04 07:30:00,0.349537
1997-01-05 07:30:00,0.268519


In [4]:
df_input.to_csv(DIRECTORY + '/examples/eerste/input.csv')
print(f'Input data printed the file at: {DIRECTORY + "/examples/eerste/input.csv"}.')

Input data printed the file at: /Users/johnkucharski/Documents/source/functionalflows/functionalflows/functionalflows//examples/eerste/input.csv.


### Historical data

Functional flow componenents and component metric values are computed based on the historical simulation data below.

In [5]:
df_historical = pd.read_csv(DIRECTORY + '/examples/eerste/historical_inflow.csv', parse_dates=['date'], index_col='date')
df_historical = df_historical[['historical_inflow']]
df_historical['historical_inflow'] = df_historical['historical_inflow'].replace(-9999.0, np.nan)
df_historical['flow'] = df_historical['historical_inflow'].apply(liters_per_day_to_m3_per_second)
df_historical.dropna(inplace=True)
df_historical.head()

,historical_inflow,flow
date,,
1989-07-01,128390400.0,1.486
1989-07-02,353289600.0,4.089
1989-07-03,277689600.0,3.214
1989-07-04,310348800.0,3.592
1989-07-05,360288000.0,4.170


## Dry season baseflow component.

This example component has the following characteristics.
    
* Timing: [01 Dec, 01 May], corresponds with dry season.
* Magnitude: exceeds the 10th percentile of historical period, 1 day flow values.
* Duration: less than 7 days. *NOTE*: this is defined as an *"anti-duration"*, we are interested in periods where timing criteria is met but the magnitude duration is NOT met for 7 or more days. This cooresponds with the timing, magnitude pattern: [1, 0] (i.e. [1=timing metric is met, 0=magnitude threshold is not met]).

To score this component we identify periods in which the flow between 01 Dec - 01 May falls below the 10th percentile of the dry season flow in the historical period for 7 or more days for this, e.g. we identify periods in which the functional flow component critieria is NOT met. This is described by the following scoring criteria: [1,0,1]. 

This must be matched with a biotic or abiotic ecosystem function or aquatic community need. For instance, in this case perhaps the needs of large bodied fish or aquatic macroinvertibrates.

In [6]:
timing = [day_of_water_year(pd.to_datetime("1-1-2023"), start=first_day_of_wy), 366]

df_historical['dry_season'] = df_historical.index.map(lambda t: 1 if timing[0] <= t.day_of_year < timing[1] else 0)
magnitude = df_historical.loc[df_historical['dry_season'] == 1]['flow'].quantile(0.10)

duration = 7
duration_pattern = np.array([1,0])

print(f'timing: {timing} days of water year.')
print(f'magnitude: > {magnitude} cms.')
print(f'duration: > {duration} consecutive days, when the timing duration pattern {duration_pattern}, respectively.')
print(f'scoring criteria: timing, magnitude, duration: [1, 0, 1] (where 1 indicates success and 0 indicates failure to meet the criteria).')

timing: [245, 366] days of water year.
magnitude: > 0.011 cms.
duration: > 7 consecutive days, when the timing duration pattern [1 0], respectively.
scoring criteria: timing, magnitude, duration: [1, 0, 1] (where 1 indicates success and 0 indicates failure to meet the criteria).


In [7]:
data = {
    'first_day_of_water_year': first_day_of_wy,
    'components': {
        'dry_season_baseflow': {
            'characteristics': ['timing', 'magnitude', 'duration'],
            'parameters': [[245, 366], [1, 0.011, '>'], [7, [1,0], '>']],
            'scoring_pattern': [1, 0, 1]
        },
    }
}
with open(DIRECTORY + 'examples/eerste/eerste.toml', 'wb') as f:
    tomli_w.dump(data, f)

## November Pulse Flow

This example component has the following characteristics.
    
* Timing: [01 Nov, 01 Dec], corresponds with a peak in the 7, 14, 30 day flow.
* Magnitude: exceeds the 75th percentile of historical November 1 day flow values.
* Rate of Change: and must represent at least a doubling of the previous days flow value (e.g. it is peak not just a large baseflow).

So, the flow between 01 Nov - 01 Dec must exceed the 75th percentile of daily flow values for November and must represent at least a doubling of the previous 1-day flow value.

This must be matched with a biotic or abiotic ecosystem function or aquatic community need. For instance, in this case perhaps this flow provides a signal for migratory species.

In [8]:
timing = [day_of_water_year(pd.to_datetime("11-1-2023"), start=first_day_of_wy), day_of_water_year(pd.to_datetime("12-1-2023"), start=first_day_of_wy)]

df_historical['november'] = df_historical.index.map(lambda t: 1 if timing[0] <= t.day_of_year < timing[1] else 0)
magnitude = df_historical.loc[df_historical['november'] == 1]['flow'].quantile(0.75)

rate_of_change = 2

print(f'timing: {timing} days of water year.')
print(f'magnitude: > {magnitude}.')
print(f'rate_of_change: > {rate_of_change}.')
print(f'scoring_pattern: [1, 1, 1]')

timing: [185, 215] days of water year.
magnitude: > 2.256.
rate_of_change: > 2.
scoring_pattern: [1, 1, 1]


In [9]:
data = {
    'first_day_of_water_year': first_day_of_wy,
    'components': {
        'dry_season_baseflow': {
            'characteristics': ['timing', 'magnitude', 'duration'],
            'parameters': [[245, 366], [1, 0.011, '>'], [7, [1,0], '>']],
            'scoring_pattern': [1, 0, 1]
        },
        'november_pulse_flow': {
            'characteristics': ['timing', 'magnitude', 'rate_of_change'],
            'parameters': [[185, 215], [1, 2.256, '>'], [1, 2.0, '>']],
            'scoring_pattern': [1, 1, 1] 
        },
    }
}
with open(DIRECTORY + 'examples/eerste/eerste.toml', 'wb') as f:
    tomli_w.dump(data, f)

## Bankfull flow

